In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
import pickle

# Load the saved SVM model and label encoder
with open('svm_model_with_encoder.pkl', 'rb') as f:
    data = pickle.load(f)
    model = data['model']
    encoder = data['encoder']

# Initialize the FaceNet embedder
embedder = FaceNet()

# Initialize the MTCNN face detector
detector = MTCNN()

# Function to get face embedding
def get_embedding(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)  # Expand dimensions for batch processing
    yhat = embedder.embeddings(face_img)  # Get face embeddings
    return yhat[0]  # Return the first (and only) embedding

# Start capturing video from the camera
cap = cv2.VideoCapture(0)  # 0 is the ID for the default camera

# Live feed loop
while True:
    ret, frame = cap.read()  # Read a frame from the camera
    if not ret:
        break
    
    # Convert the frame to RGB (since OpenCV uses BGR)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces using MTCNN
    faces = detector.detect_faces(rgb_frame)
    
    for face in faces:
        # Get the bounding box of the face
        x, y, w, h = face['box']
        x, y = abs(x), abs(y)  # Ensure positive coordinates
        face_img = rgb_frame[y:y+h, x:x+w]  # Crop the face from the frame
        
        try:
            # Resize the face to 160x160 as expected by FaceNet
            face_img = cv2.resize(face_img, (160, 160))
            
            # Get the face embedding
            embedding = get_embedding(face_img)
            
            # Predict using the trained SVM model
            embedding = np.expand_dims(embedding, axis=0)  # Expand dimensions for the SVM
            prediction = model.predict(embedding)
            probability = model.predict_proba(embedding)

            # Decode the predicted class label
            class_label = encoder.inverse_transform(prediction)[0]
            
            # Draw a rectangle around the face and display the name
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_label} ({probability.max()*100:.2f}%)', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        except Exception as e:
            # In case of any issue (e.g., face detection failure), continue
            print(f"Error processing face: {e}")
            continue
    
    # Display the frame with the bounding boxes and labels
    cv2.imshow('Face Recognition', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━